In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Essential_Thrombocythemia/GSE57793'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression data from patients with Essential Thrombocythemia (ET), Polycythemia Vera (PV), Primary Myelofibrosis (PMF)"
!Series_summary	"Microarrays were used to assess gene expression in patients with ET, PV, and PMF before and after treatment with IFNalpha2 in a paired design."
!Series_overall_design	"Total RNA was purified from whole blood and amplified to biotin-labeled aRNA and hybridized to microarray chips. Background correction, normalization, and gene expression index calculation were performed with the robust multi-array (rma) method. The regularized t-test limma for pairwise data was used to calculate differences in gene expression between patients before and after treatment with IFN-alpha2."
Sample Characteristics Dictionary:
{0: ['disease state: ET', 'disease state: PMF', 'disease state: PV'], 1: ['treatment: untreated', 'treatment: IFN-alpha2'], 2: ['tissue: Whole blood']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determining if gene expression data is available
is_gene_available = True  # Because the dataset involves microarrays for gene expression

# Determining data availability
sample_characteristics = {0: ['disease state: ET', 'disease state: PMF', 'disease state: PV'], 
    1: ['treatment: untreated', 'treatment: IFN-alpha2'], 2: ['tissue: Whole blood']}

# Based on the available data, only 'disease state' seems close to 'Essential_Thrombocythemia'
if len(set(sample_characteristics[0])) > 1:
    trait_row = 0
else:
    trait_row = None

# There does not appear to be any explicit age or gender data in the sample characteristics dictionary
age_row = None
gender_row = None

# Define conversion functions
def convert_trait(value):
    mapping = {'ET': 1, 'PMF': 0, 'PV': 0}
    return mapping.get(value.split(': ')[1], None)

def convert_age(value):
    try:
        return int(value.split(': ')[1])
    except:
        return None

# This would be the placeholder as there is no gender data available
def convert_gender(value):
    mapping = {'female': 0, 'male': 1}
    return mapping.get(value.split(': ')[1].lower(), None)

# Save cohort information
save_cohort_info('GSE57793', './preprocessed/Essential_Thrombocythemia/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction if trait_row is not None
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Essential_Thrombocythemia', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Essential_Thrombocythemia/trait_data/GSE57793.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1388566': [1], 'GSM1388567': [1], 'GSM1388568': [1], 'GSM1388569': [1], 'GSM1388570': [1], 'GSM1388571': [1], 'GSM1388572': [1], 'GSM1388573': [1], 'GSM1388574': [1], 'GSM1388575': [1], 'GSM1388576': [1], 'GSM1388577': [1], 'GSM1388578': [1], 'GSM1388579': [1], 'GSM1388580': [1], 'GSM1388581': [1], 'GSM1388582': [0], 'GSM1388583': [0], 'GSM1388584': [0], 'GSM1388585': [0], 'GSM1388586': [0], 'GSM1388587': [0], 'GSM1388588': [0], 'GSM1388589': [0], 'GSM1388590': [0], 'GSM1388591': [0], 'GSM1388592': [0], 'GSM1388593': [0], 'GSM1388594': [0], 'GSM1388595': [0], 'GSM1388596': [0], 'GSM1388597': [0], 'GSM1388598': [0], 'GSM1388599': [0], 'GSM1388600': [0], 'GSM1388601': [0], 'GSM1388602': [0], 'GSM1388603': [0], 'GSM1388604': [0], 'GSM1388605': [0], 'GSM1388606': [0], 'GSM1388607': [0], 'GSM1388608': [0], 'GSM1388609': [0], 'GSM1388610': [0], 'GSM1388611': [0], 'GSM1388612': [0], 'GSM1388613': [0], 'GSM1388614': [0], 'GSM1388615': [0], 'GSM1388616': [0], 'GSM1388617': [0], 'GSM1388618

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant keys based on the previews from the previous steps
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Essential_Thrombocythemia/gene_data/GSE57793.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Essential_Thrombocythemia')

# 4. Save the cohort information.
save_cohort_info('GSE57793', './preprocessed/Essential_Thrombocythemia/cohort_info.json', True, True, trait_biased, unbiased_merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Essential_Thrombocythemia/GSE57793.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Essential_Thrombocythemia', the least common label is '1.0' with 16 occurrences. This represents 24.24% of the dataset.
The distribution of the feature 'Essential_Thrombocythemia' in this dataset is fine.

